# Projet 8 - Openclassrooms - Déployer un modèle dans le cloud

## Import

In [8]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row, SparkSession, SQLContext

import io
from io import StringIO
import os
import sys
import time

import numpy as np
import pandas as pd

import cv2
from PIL import Image


import boto.s3
import boto3


import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark


# Installation de sparkdl

In [18]:
pip install sparkdl

     |████████████████████████████████| 99 kB 7.7 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install Keras

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

     |████████████████████████████████| 394.3 MB 7.1 kB/s  eta 0:00:01
     |████████████████████████████████| 127 kB 74.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 70.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 10.6 MB 58.3 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 74.6 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 74.6 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 71.7 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 52.6 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 52.6 MB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=b71f982c38a08f7339d06f3980271051044a12a7d519732d35cae8d3c5416a50
  Stored in direc

In [2]:
pip install tensorframes

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install kafka

     |████████████████████████████████| 207 kB 19.4 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflowonspark

     |████████████████████████████████| 44 kB 3.2 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install jieba

     |████████████████████████████████| 19.2 MB 18.2 MB/s eta 0:00:01
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314477 sha256=3815e2da9554db8b23245cc90d84bf7167c8b68ed0838ffdd07e942709e3d333
  Stored in directory: /home/ec2-user/.cache/pip/wheels/17/a7/8b/a7e03881534e78558920ac68aaeca05180c0e2c3d11c4fce3b
Successfully built jieba
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Importation DeepImageFeaturizer

In [9]:
from sparkdl import DeepImageFeaturizer

# Définition des fonctions de pré-processing

In [111]:
# Renvoie la catéagorie d'une image à partir de son chemin  
# Entrée : chaine de caractères, contient le chemin complet de l'image 
# Sortie: tuple de chaines de caratères  catégorie, nom du fichier

def parse_categorie(path):
    if len(path) > 0:
        return path.split('/')[-1],path.split('/')[-2],
    else:
        return 'None','None'
    

# Renvoie la liste des descripteurs ORB d'une image format binaire
# Entrée: image lue en binaire dans le rdd
# Sortie: tuple categorie, tableau de descripteurs

def cat_image_np(imgfile_imgbytes):

    imgfilename, imgbytes = imgfile_imgbytes
    #print(imgfilename)
        #print(imgbytes)
    nom, categorie = parse_categorie(imgfilename)
        #print(categorie)
        # convertion  binaire en RGB image
    img2 = Image.open(io.BytesIO(imgbytes))
        #print(img2)
        #conversion  Image en tableau  numpy
        #img_np = np.array(img2)
    img_np = img_to_array(img2)
        #print(img_np)
        #orb = cv2.ORB_create()
        #kp, descriptors = orb.detectAndCompute(img_np, None)

    return Row(fileName=nom,category=categorie, image=preprocess_input(img_np))

    
# Renvoie un histogramme de fréquence du BOVW
# Entrée: Ligne d'un RDD contenant la liste des descripteurs d'une image
# Entrée: modele K-means entrainé sur l'ensemble des descripteurs
# Entrée: nombre de classes prédites par K-means
# Sortie: Ligne de RDD contenant nom (chaine), catégorie (chaine), vecteur sous forme d'une liste de 32 reels  
def bow(row, model,k):
    #entree: tables de descripteurs
    categorie = row['category']
    nom = row['name']
    feature_matrix = row['descriptors']
    bow = np.zeros(k)
    nkp = len(row['descriptors'])
    #print(nkp)

    for x in feature_matrix:
        idx = model.predict(x)
        #print(idx)
        bow[idx] += 1/nkp

    return Row(fileName=nom,category=categorie, bow=bow.tolist())


# Ecrit un dataframe spark au format parket
# Entrée: dataframme spark, chemin (chaine)
# Sortie: sauvegarde format parquet sur l'emplacement indiqué

def write_dataframe_parquet(dataframe, path_parquet):
    '''Enregistrement du spark dataframe au format parquet au chemin specifie'''
    try:
        #start = time.time()
        dataframe.write.format("parquet").mode('overwrite').save(path_parquet)
        print('Enregistrement effectué.')
            

        #print('Temps de sauvegarde : {} secondes'.format(time.strftime('%S', time.gmtime(time.time()-start))))
    except:
        print('L\'Enregistrement a échoué.')
    
    return True

In [18]:
def preprocess_data(dataframe):
    '''Renvoie le résultat de l'avant dernière couche de chaque image du dataframe via transform du ResNet50
    return un df contenant des vecteurs de dimension 1x2048 '''
    
    from sparkdl import DeepImageFeaturizer
    # DeepImageFeaturizer Applies the model specified by its popular name, 
    # with its prediction layer(s) chopped off
    featurizer = DeepImageFeaturizer(inputCol="image", outputCol="image_preprocessed", modelName="ResNet50")
    output = featurizer.transform(dataframe).select(['path', 'categorie', 'image_preprocessed'])
    del featurizer
    return output

In [42]:
import boto3

cn_regions = ['cn-north-1', 'cn-northwest-1']
region = boto3.Session().region_name
endpoint_domain = 'com.cn' if region in cn_regions else 'com'
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 's3.{}.amazonaws.{}'.format(region, endpoint_domain))

trainingData = spark.read.format('libsvm')\
    .option('numFeatures', '784')\
    .load('s3a://sagemaker-sample-data-{}/spark/mnist/train/'.format(region))

testData = spark.read.format('libsvm')\
    .load('s3a://sagemaker-sample-data-{}/spark/mnist/test/'.format(region))

testData.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  7.0|(778,[202,203,204...|
|  2.0|(778,[94,95,96,97...|
|  1.0|(778,[128,129,130...|
|  0.0|(778,[124,125,126...|
|  4.0|(778,[150,151,159...|
|  1.0|(778,[156,157,158...|
|  4.0|(778,[149,150,151...|
|  9.0|(778,[179,180,181...|
|  5.0|(778,[129,130,131...|
|  9.0|(778,[209,210,211...|
|  0.0|(778,[123,124,125...|
|  6.0|(778,[94,95,96,97...|
|  9.0|(778,[208,209,210...|
|  0.0|(778,[152,153,154...|
|  1.0|(778,[125,126,127...|
|  5.0|(778,[124,125,126...|
|  9.0|(778,[179,180,181...|
|  7.0|(778,[200,201,202...|
|  3.0|(778,[118,119,120...|
|  4.0|(778,[158,159,185...|
+-----+--------------------+
only showing top 20 rows



# Définition de l'environement Spark

In [10]:
role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
    .master("local[*]").getOrCreate()
    
spark

# Variables globales

In [11]:
# Chemin d'acces des données sur S3
# Seul un échantillon des données de base est utilisé

sample_img_dir = "s3a://p8muriel/test/*"
path_write = "s3a://p8muriel/test_deep_features.parquet"

# Pré-processing

## Chargement des données

Un échantillon des données sont stockées sur un bucket "p8muriel" sur S3. Ni les chemins, ni les noms de fichiers ne doivent contenir des espaces.  
-> faire un script préalable au chargement des données sur S3  

In [12]:
print("Chargement des images")
#Chargement des images dans un RDD sous forme binaire
t = time.time()
images = spark.sparkContext.binaryFiles(sample_img_dir)
print('...Temps de chargement des images : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

Chargement des images
...Temps de chargement des images : 00:00:01


In [57]:
from sparkdl import DeepImageFeaturizer

In [61]:
images.take(2)

[('s3a://p8muriel/test/AppleRed3/44_100.jpg',
  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x00d\x00d\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x8

In [112]:
images_proc = images.map(lambda img: cat_image_np(img))

In [93]:
  model = ResNet50(weights=None, include_top=False)
  model.set_weights(bc_model_weights.value)

In [126]:
x = np.expand_dims(t[2], -1)

In [52]:
bc_model_weights = spark.sparkContext.broadcast(model.get_weights())

def model_fn():
  """
  Returns a ResNet50 model with top layer removed and broadcasted pretrained weights.
  """
  model = ResNet50(weights=None, include_top=False)
  model.set_weights(bc_model_weights.value)
  return model

In [136]:
from tensorflow.keras.preprocessing import image

In [137]:
img = image.load_img("r_87_100.jpg", target_size=(244, 244))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [140]:
test=model.predict(x)

In [141]:
test.shape

(1, 8, 8, 2048)

In [144]:
test[0]

array([[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

       [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 4.2344694e+00, ...,
         1.947

In [53]:
def preprocess(content):
  """
  Preprocesses raw image bytes for prediction.
  """
  img = Image.open(io.BytesIO(content))
  arr = img_to_array(img)
  return preprocess_input(arr)

def featurize_series(model, content_series):
  """
  Featurize a pd.Series of raw images using the input model.
  :return: a pd.Series of image features
  """
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
  output = [p.flatten() for p in preds]
  return pd.Series(output)

In [55]:
#@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
  '''
  This method is a Scalar Iterator pandas UDF wrapping our featurization function.
  The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
  :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
  '''
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)

In [56]:
# Pandas UDFs on large records (e.g., very large images) can run into Out Of Memory (OOM) errors.
# If you hit such errors in the cell below, try reducing the Arrow batch size via `maxRecordsPerBatch`.
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [ ]:
# We can now run featurization on our entire Spark DataFrame.
# NOTE: This can take a long time (about 10 minutes) since it applies a large model to the full dataset.
features_df = images.repartition(16).select(col("path"), featurize_udf("content").alias("features"))
features_df.write.mode("overwrite").parquet("dbfs:/ml/tmp/flower_photos_features")

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [2]:
import tensorflow as tf

In [24]:
sess = tf.Session()

In [25]:
sess

In [26]:
sess = tf.compat.v1.Session()

In [31]:
import pandas as pd
from PIL import Image
import numpy as np
import io

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

from pyspark.sql.functions import col, pandas_udf, PandasUDFType

# Formation du bag of visual word

In [34]:
model = ResNet50(include_top=False)
model.summary()

94773248/94765736 [==============================] - 3s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
__________________________

In [21]:
df = cat_image.toDF()

In [43]:
df2 = images.toDF()

TypeError: not supported type: <class 'bytes'>

In [52]:
#Clustering des descripteurs avec KMEANS
print("Clustering des descripteurs")

Clustering des descripteurs


In [53]:
#calcul du nombre de classe avec la formule nombre de categories x 10
print("...calcul du nombre de classes")
t = time.time()
nbcat = cat_descriptors.map(lambda x: x['category']).distinct().count()
k = nbcat *10
print("K-means avec k = ",k)
print('.....Temps de traitement count : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...calcul du nombre de classes
K-means avec k =  150
.....Temps de traitement count : 00:10:42


In [54]:
print("...flatmap des descripteurs dans un rdd")
t = time.time()
# rdd contenant un descripteur par ligne
cat_features = tuples_decriptors.flatMap(lambda x: [(x[0],x[1],w) for w in x[2]])
cat_desc = cat_features\
    .map(lambda x: (Row(name=x[0],category=x[1], features=x[2])))
descriptors = cat_desc.map(lambda x: x['features'])
print('.....Temps de traitement flatmap : {} '.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...flatmap des descripteurs dans un rdd
.....Temps de traitement flatmap : 00:00:00 


In [55]:
print('...clustering des descripteurs de kMeans')
t = time.time()      
#entrainement du modele
model = KMeans.train(descriptors, k, maxIterations=10, initializationMode="random")
print('.....Temps de traitement kMeans : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...clustering des descripteurs de kMeans
.....Temps de traitement kMeans : 00:09:52


In [35]:
print("...formation du rdd bag of visual word")
t = time.time()    
rdd_bow = cat_descriptors.map(lambda x: bow(x,model,k))
print('.....Temps de traitement bow : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...formation du rdd bag of visual word
.....Temps de traitement bow : 00:00:00


## Affichage du contenu du RDD

Juste pour information dans le cadre du projet, à ne pas faire en production

In [45]:
rdd_bow.take(2)

[Row(bow=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.043478260869565216, 0.021739130434782608, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.043478260869565216, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021739130434782608, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.043478260869565216, 0.0, 0.043478260869565216, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23913043478260865, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021739130434782608, 0.021739130434782608, 0.021739130434782608, 0.0, 0.0, 0.043478260869565216, 0.0, 0.0, 0.021739130434782608, 0.021739130434782608, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.043478260869565216, 0.021739130434782608, 0.0, 0.0, 0.0, 0.0, 0.043478260869565216, 0.0, 0.021739130434782608, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021739130434782608, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06521739130434782, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021739130434782608, 0.02173

# Enregistrement des données au format parquet

In [36]:
print("...conversion du rdd BOVW en spark dataframe")
t = time.time()  
df = rdd_bow.toDF()
print('.....Temps de traitement toDF : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...conversion du rdd BOVW en spark dataframe
.....Temps de traitement toDF : 00:00:02


In [46]:

print("...enregistrement format parquet")
t = time.time()  
df = rdd_bow.toDF()

#path_write = "s3a://p8muriel/test_bovw.parquet"

write_dataframe_parquet(df,path_write)
print('.....Temps de traitement enregistrement parquet : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...enregistrement format parquet
Enregistrement effectué.
.....Temps de traitement enregistrement parquet : 00:10:13


# Aperçu des données après réduction de dimension

In [37]:
df.show()

+--------------------+--------+-----------+
|                 bow|category|   fileName|
+--------------------+--------+-----------+
|[0.0, 0.0, 0.0, 0...|   Salak|  0_100.jpg|
|[0.0, 0.047619047...|   Salak|100_100.jpg|
|[0.0, 0.047619047...|   Salak|101_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|102_100.jpg|
|[0.0, 0.055555555...|   Salak|103_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|104_100.jpg|
|[0.0, 0.043478260...|   Salak|105_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|106_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|107_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|108_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|109_100.jpg|
|[0.0, 0.076923076...|   Salak|110_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|111_100.jpg|
|[0.0, 0.071428571...|   Salak|112_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|113_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|114_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|115_100.jpg|
|[0.0, 0.0, 0.0416...|   Salak|116_100.jpg|
|[0.0, 0.0, 0.0, 0...|   Salak|117_100.jpg|
|[0.0, 0.029411764...|   Salak|1

In [38]:
df.show(2, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Schema

In [44]:
df.schema

StructType(List(StructField(bow,ArrayType(DoubleType,true),true),StructField(category,StringType,true),StructField(fileName,StringType,true)))

In [42]:
df.printSchema()

root
 |-- bow: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- category: string (nullable = true)
 |-- fileName: string (nullable = true)



## Ecriture en csv pour comparaison avec parquet

In [49]:
print("...enregistrement format csv")
t = time.time() 
df.toPandas().to_csv("s3a://p8muriel/test2_bow_file.csv", header=True)
print('.....Temps de traitement enregistrement csv : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...enregistrement format csv
.....Temps de traitement enregistrement csv : 00:10:02


In [64]:
spark.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

In [62]:
spark.app.name

AttributeError: 'SparkSession' object has no attribute 'app'